In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


## **Load Dates from Shortcut Bronze table, encrich with columns, save to Silver table**

In [1]:
dfBronzeDates = spark.read.format("delta").load("Tables/bronze_dates")

StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 3, Finished, Available)

In [ ]:
from pyspark.sql.functions import concat,concat_ws,col,lpad,substring,lit,to_date

In [5]:
display(dfBronzeDates \
    .withColumn("Date", to_date("Date")) \
    .select("*", \
        concat("Year", lpad("Month", 2,"0")).alias("MonthOfYearNumber"), \
        concat("Year", lpad("Quarter", 2,"0")).alias("QuarterOfYearNumber"), \
        concat_ws(" ", substring("MonthName", 1,3), "Year").alias("MonthOfYear"), \
        concat(lit("Q"), "Quarter", lit(" "), "Year").alias("QuarterOfYear")) )

StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8f9e44a1-3fa7-47ab-af04-4fbcac629538)

In [6]:


dfBronzeDates \
    .withColumn("Date", to_date("Date")) \
    .select("*", \
        concat("Year", lpad("Month", 2,"0")).alias("MonthOfYearNumber"), \
        concat("Year", lpad("Quarter", 2,"0")).alias("QuarterOfYearNumber"), \
        concat_ws(" ", substring("MonthName", 1,3), "Year").alias("MonthOfYear"), \
        concat(lit("Q"), "Quarter", lit(" "), "Year").alias("QuarterOfYear")) \
    .write.mode("overwrite").format("delta").save("Tables/" + "silver_dates")


StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 8, Finished, Available)

## **Load shortcut Bronze NYC Yellow taxi data to Dataframe**

In [8]:
dfBronzeTaxi = spark.read.format("delta").load("Tables/bronze_nyctlcYellow")

StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 9, Finished, Available)

## **Load from Lakehouse Files area to Bronze Table then run Optimize to compact files**

In [9]:
from pyspark.sql.functions import to_date

dfBronzeTaxi \
    .select("*", \
        to_date("tpepPickupDateTime").alias("tpepPickupDate"), \
        to_date("tpepDropoffDateTime").alias("tpepDropoffDate")) \
    .write.mode("overwrite").format("delta").save("Tables/" + "silver_nyctlcYellow")


StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 10, Finished, Available)

In [10]:
%%sql

OPTIMIZE silver_nyctlcYellow

StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 11, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

In [12]:
%%sql

select min(tpepPickupDateTime), max(tpepPickupdateTime), count(1) from silver_nyctlcYellow

StatementMeta(, 44779f79-14db-4786-8eb3-cf3fb47a1a48, 13, Finished, Available)

<Spark SQL result set with 1 rows and 3 fields>